In [1]:
import pandas as pd, sqlite3 as lite, numpy as np

# '2017.8.29-2017.9.30职员销售明细表.xls'
df = pd.read_excel('data\\2017.10.1-2017.10.10职员销售明细表.xls.xls',sheetname='2017.10.1-2017.10.10职员销售明细表.xls',index_col=0)
df['职员名称'] = None
print(df.columns)
print(len(df))
dfdel = df[(df.单位全名.isnull().values==True) & ((df.单据编号.isnull().values == True) |  (df.单据编号 =='小计') |  (df.单据编号 =='合计'))]
hangdel = list(dfdel.index)
print(hangdel)
print(dfdel)
df1 = df.drop(hangdel)
print(df1.columns)
print(len(df1))
# print(df1)
dfzhiyuan = df1[df1.单位全名.isnull().values == True]
print(len(dfzhiyuan))
print(dfzhiyuan)
zyhang = list(dfzhiyuan.index)
zyming = list(dfzhiyuan['单据编号'])
for i in range(len(zyhang)):
    df.loc[zyhang[i]:,'职员名称'] = zyming[i]
dfdel = df[df.单位全名.isnull().values==True]
hangdel = list(dfdel.index)
print(hangdel)
print(dfdel)
df2 = df.drop(hangdel)
df2['日期'] = pd.to_datetime(df2['日期'])
# df2['备注'] = str(df2['备注'])
# df2['型号'] = str(df2['型号'])
# print(df2.dtypes)
# print(df2.columns)
# print(len(df2))
# print(df2)
df2.index = range(len(df2))
df=df2.loc[:,['日期','单据编号','单据类型','职员名称','职员销售明细表','备注','商品备注','商品编号','商品全名','单价','单位','数量','金额','单位全名','仓库全名','部门全名']]
# del df[行号]
print(df)
# cnx = lite.connect('data\\quandan.db')
# df.to_sql(name='xiaoshoumingxi', con=cnx, if_exists='append', chunksize=10000)

Index(['日期', '单据编号', '摘要', '单据类型', '备注', '商品备注', '商品编号', '商品全名', '规格', '型号',
       '产地', '单位', '数量', '单价', '金额', '含税单价', '价税合计', '成本金额', '毛利', '毛利率',
       '单位全名', '仓库全名', '部门全名', '职员名称'],
      dtype='object')
3898
[915, 916, 1125, 1126, 1711, 1712, 2533, 2534, 2705, 2706, 3009, 3010, 3581, 3582, 3896, 3897, 3898]
       日期 单据编号   摘要 单据类型  备注 商品备注 商品编号 商品全名   规格  型号  ...          金额 含税单价  \
行号                                                    ...                    
915   NaN   小计  NaN  NaN NaN  NaN  NaN  NaN  NaN NaN  ...    20798.46  NaN   
916   NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN NaN  ...         NaN  NaN   
1125  NaN   小计  NaN  NaN NaN  NaN  NaN  NaN  NaN NaN  ...    64627.87  NaN   
1126  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN NaN  ...         NaN  NaN   
1711  NaN   小计  NaN  NaN NaN  NaN  NaN  NaN  NaN NaN  ...    14019.75  NaN   
1712  NaN  NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN NaN  ...         NaN  NaN   
2533  NaN   小计  NaN  NaN NaN  NaN  NaN  NaN  NaN NaN  ...